In [633]:
import pandas as pd
import seaborn as sns
import datetime
from datetime import *
import sys
import gzip
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pylab
import scipy.stats as stats

In [634]:
plt.style.use('seaborn')
pd.options.mode.chained_assignment = None  # default='warn'

# beer

In [635]:
#load the data beers.csv
df_beers = pd.read_csv('./data/beers.csv')

In [636]:
df_beers.head()

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,3.8,NaN,NaN,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.8,NaN,NaN,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.0,NaN,NaN,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.5,NaN,NaN,0,NaN


In [637]:
list_considered_variable=['beer_id','beer_name','brewery_id','brewery_name','style',
                          'nbr_reviews','nbr_ratings','avg', 'abv']
df_beers=df_beers[list_considered_variable]

In [638]:
beer_catalog = {
    'Bocks' : ['Bock', 'Doppelbock', 'Eisbock', 'Maibock', 'Weizenbock', 'Maibock / Helles Bock'],

    'Dark Ales' : ['Dubbel', 'Roggenbier', 'Scottish Ale', 'Winter Warmer'],

    'Hybrid Beers' : ['Bière de Champagne / Bière Brut', 'Braggot', 'California Common / Steam Beer', 'Cream Ale'],

    'Pale Ales' : ['American Amber / Red Ale', 'American Blonde Ale', 'American Pale Ale', 'Belgian Blonde Ale',
    'Belgian Pale Ale', 'Bière de Garde', 'English Bitter', 'English Pale Ale', 'English Pale Mild Ale', 'Extra Special / Strong Bitter (ESB)',
    'Grisette', 'Irish Red Ale', 'Kölsch', 'Saison', 'Saison / Farmhouse Ale', 'Belgian Strong Pale Ale', 'American Pale Ale (APA)'],

    'Porters' : ['American Porter', 'Baltic Porter', 'English Porter', 'Imperial Porter', 'Robust Porter ', 'Smoked Porter'],

    'Stouts' : ['American Imperial Stout', 'American Stout', 'English Stout', 'Foreign / Export Stout', 'Irish Dry Stout', 'Oatmeal Stout',
    'Russian Imperial Stout', 'Milk / Sweet Stout', 'American Double / Imperial Stout'],

    'Wheat Beers' : ['American Dark Wheat Beer', 'American Pale Wheat Beer', 'Dunkelweizen', 'Grodziskie', 'Hefeweizen', 'Kristallweizen', 'Witbier', 
    'American Pale Wheat Ale', 'American Dark Wheat Ale', 'Berliner Weissbier', 'Kristalweizen'],

    'Brown Ales' : ['Altbier', 'American Brown Ale', 'Belgian Dark Ale', 'English Brown Ale', 'English Dark Mild Ale', 'Belgian Strong Dark Ale'],

    'Dark Lagers' : ['American Amber / Red Lager', 'Czech Amber Lager', 'Czech Dark Lager', 'European Dark Lager', 'Märzen',
    'Munich Dunkel', 'Rauchbier', 'Schwarzbier', 'Vienna Lager', 'Munich Dunkel Lager', 'Euro Dark Lager'],

    'India Pale Ales' : ['American IPA', 'American Double / Imperial IPA', 'Belgian IPA', 'Black IPA', 'Brut IPA', 'English IPA', 'English India Pale Ale (IPA)', 
    'Imperial IPA', 'Milkshake IPA', 'New England IPA', 'American Black Ale'],

    'Pale Lagers' : ['American Adjunct Lager', 'American Lager', 'Bohemian / Czech Pilsner', 'Czech Pale Lager', 'European / Dortmunder Export Lager',
    'European Pale Lager', 'European Strong Lager', 'Festbier / Wiesnbier', 'German Pilsner', 'Helles', 'Imperial Pilsner', 'India Pale Lager (IPL)',
    'Kellerbier / Zwickelbier', 'Light Lager', 'Malt Liquor', 'Czech Pilsener', 'Euro Pale Lager', 'American Pale Lager', 'German Pilsener', 'Dortmunder / Export Lager', 
    'Euro Strong Lager', 'Munich Helles Lager', 'American Double / Imperial Pilsner', 'American Malt Liquor', 'Märzen / Oktoberfest'],

    'Specialty Beer' : ['Chile Beer', 'Fruit and Field Beer', 'Gruit / Ancient Herbed Ale', 'Happoshu', 'Herb and Spice Beer', 'Japanese Rice Lager',
    'Kvass', 'Low-Alcohol Beer', 'Pumpkin Beer', 'Rye Beer', 'Sahti', 'Smoked Beer', 'Herbed / Spiced Beer', 'Pumpkin Ale', 'Scottish Gruit / Ancient Herbed Ale', 'Low Alcohol Beer'],

    'Strong Ales' : ['American Barleywine', 'American Strong Ale', 'Belgian Dark Strong Ale', 'Belgian Pale Strong Ale', 'English Barleywine', 'English Strong Ale',
    'Imperial Red Ale', 'Old Ale', 'Quadrupel (Quad)', 'Scotch Ale / Wee Heavy', 'Tripel', 'Wheatwine', 'Fruit / Vegetable Beer'],
    
    'Wild/Sour Beers' : ['Berliner Weisse', 'Brett Beer', 'Faro', 'Flanders Oud Bruin', 'Flanders Red Ale', 'Fruit Lambic',
    'Fruited Kettle Sour', 'Gose', 'Gueuze', 'Lambic', 'Wild Ale', 'Lambic - Unblended', 'Lambic - Fruit', 'American Wild Ale'],

    'Weird Cocktail' : ['Black & Tan']
}

In [639]:
beer_catalog_reverse = {}
for key, values in beer_catalog.items():
    for value in values:
            beer_catalog_reverse.setdefault(value, []).append(key)

In [640]:
def category_check(beer_style, beer_catalog):
    '''
    Return the new columns of 15 categories made in the website https://www.beeradvocate.com/beer/styles/
    '''
    for category_name, beer_styles in beer_catalog.items():
        if beer_style in beer_styles:
            return category_name

In [641]:
df_beers['category'] = df_beers['style'].apply(lambda beer_style: category_check(beer_style, beer_catalog))

In [642]:
df_beers

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_reviews,nbr_ratings,avg,abv,category
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.7,Pale Lagers
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,3.8,Pale Lagers
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.8,Pale Lagers
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.0,Pale Lagers
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.5,Pale Lagers
...,...,...,...,...,...,...,...,...,...,...
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,3,2.71,5.0,Pale Ales
280819,19140,Nut Brown Ale,885,Summit Station Restaurant & Brewery,English Brown Ale,1,2,3.10,5.3,Brown Ales
280820,19146,Octoberfest,885,Summit Station Restaurant & Brewery,Märzen / Oktoberfest,0,0,NaN,NaN,Pale Lagers
280821,2805,Scotch Ale,885,Summit Station Restaurant & Brewery,Scotch Ale / Wee Heavy,0,0,NaN,7.0,Strong Ales


# Users and filtering

In [643]:
#load the data

df_users = pd.read_csv('./data/users.csv')
print(df_users.describe())
df_users.head(5)

         nbr_ratings    nbr_reviews        joined
count  153704.000000  153704.000000  1.510520e+05
mean       54.605163      16.847876  1.357238e+09
std       252.388790     139.846706  9.195131e+07
min         1.000000       0.000000  8.407944e+08
25%         1.000000       0.000000  1.303121e+09
50%         3.000000       0.000000  1.391944e+09
75%        16.000000       2.000000  1.417691e+09
max     12046.000000    8970.000000  1.501495e+09


,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland


In [644]:
# First user filtering phase 

# keep users from the USA only & who have made a minimum number of reviews in order to participate in our study

df_us_users = df_users[df_users.location.str.contains('United States', na=False)]
nb_us_users = df_us_users['user_id'].nunique()
print(f"There are {nb_us_users} US users")

#choose a minimum number of ratings per user, since we want to analyse an evolution!
cutoff_low = 500
print("Cutoff low : ", cutoff_low)

#choose a maximum number of ratings per user, we want to filter our super-users or organisations
cutoff_high = df_us_users.nbr_ratings.quantile(0.99)
print("Cutoff high : ", cutoff_high)

#create a valid user dataframe and a set with the valid user_names
df_us_users = df_us_users[(df_us_users.nbr_ratings < cutoff_high) &
                                (df_us_users.nbr_ratings >= cutoff_low)]
us_users_names = set(df_us_users.user_name)
print("Number of users left : ", len(us_users_names))
print("Total of ratings left : ", df_us_users.nbr_ratings.sum())

#convert the 'joined' column into datetime format
df_us_users.loc[:,'date_joined'] = pd.to_datetime(df_us_users['joined'].apply(lambda dt : datetime.fromtimestamp(dt).strftime('%Y-%m-%d')))

There are 113223 US users
Cutoff low :  500
Cutoff high :  1135.0
Number of users left :  2232
Total of ratings left :  1652164


In [645]:
# We now gather some more user information by analyzing their behaviour as a reviewer

In [646]:
def parse(filename):
    """
    Parse a txt.gz file and return a generator for it
    Copyright © 2017 Gael Lederrey <gael.lederrey@epfl.ch>
    :param filename: name of the file
    :return: Generator to go through the file
    """
    file = gzip.open(filename, 'rb')
    entry = {}
    # Go through all the lines
    for line in file:
        # Transform the string-bytes into a string
        line = line.decode("utf-8").strip()

        # We check for a colon in each line
        colon_pos = line.find(":")
        if colon_pos == -1:
            # if no, we yield the entry
            yield entry
            entry = {}
            continue
        # otherwise, we add the key-value pair to the entry
        key = line[:colon_pos]
        value = line[colon_pos + 2:]
        entry[key] = value

In [647]:
gen = parse('./data/ratings.txt.gz')

In [648]:
#We create the dataframe of the valid us users previously determined

create = False

if create:
    undesired_features = set(['text','review', 'brewery_name', 'brewery_id', 'user_id', 'beer_name'])
    full_ratings_ls = []
    int_cols = ['beer_id']
    float_cols = ['abv','appearance', 'aroma', 'palate', 'taste', 'overall', 'rating']

    review = next(gen)
    cols = [key for key, _ in review.items() if key not in undesired_features]

    for idx, review in enumerate(gen):
        user_name = review['user_name']
        valid_rows = [value for key, value in review.items() if key not in undesired_features]
        if user_name in us_users_names:
            full_ratings_ls.append(valid_rows)

    df_ratings = pd.DataFrame(full_ratings_ls, columns = cols)

    df_ratings[int_cols] = df_ratings[int_cols].astype('int')
    df_ratings[float_cols] = df_ratings[float_cols].astype('float') 
    df_ratings["date"] = pd.to_datetime(df_ratings.date.apply(
        lambda date : datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d')
    ))

    #There is one review in 2000, wich is an outlier
    start_date = pd.to_datetime("2001-01-01")
    df_ratings = df_ratings.loc[df_ratings['date'] >= start_date]   
    df_ratings.to_pickle('./data/ratings_valid_user.pkl')

else:
    df_ratings = pd.read_pickle('./data/ratings_valid_user.pkl')

In [649]:
#add the category of the beer
beers_id_cat = df_beers[['beer_id', 'category']]
df_ratings = pd.merge(df_ratings, beers_id_cat, on = 'beer_id')

#add the rating number, starts at 0
df_ratings.sort_values(by = "date", inplace=True)
df_ratings["rating_number"] = df_ratings.groupby("user_name").transform("cumcount")

#we also convert the date to the proper format
df_ratings["date"] = pd.to_datetime(df_ratings["date"])

In [650]:
#We apply the second of our filters, we only keep the 500 first ratings. (rating number at which you are considered an expert)
nb_ratings_expert = 500
df_us_ratings = df_ratings[df_ratings["rating_number"] < nb_ratings_expert]

In [651]:
df_us_ratings.head(5)

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number
1640018,167,Witbier,NaN,2001-03-16,frank4sail,4.5,4.5,4.0,5.0,5.0,4.75,Wheat Beers,0
311428,3558,Czech Pilsener,5.00,2001-06-18,Mark,4.0,4.0,3.0,4.0,4.0,3.90,Pale Lagers,0
17313,73,Milk / Sweet Stout,5.20,2001-06-18,Mark,5.0,5.0,5.0,5.0,5.0,5.00,Stouts,1
136697,261,Vienna Lager,4.48,2001-06-24,Mark,4.0,3.0,3.0,3.0,3.0,3.06,Dark Lagers,2
442671,87,American Pale Ale (APA),5.40,2001-06-24,Mark,4.0,4.0,1.0,3.0,3.0,3.10,Pale Ales,3


In [652]:
#We go back to users and compute on how many different days they were active

user_active = df_us_ratings.groupby(['user_name'], as_index = False).date.nunique()
user_active.columns = ['user_name', 'active_days']
df_us_users = df_us_users.merge(user_active, on = 'user_name')

In [653]:
# We compute how long it took them to submit the 500 ratings
user_delta = df_us_ratings.groupby('user_name', as_index = False).apply(lambda usr: (usr.date.max() - usr.date.min()).days)
user_delta.columns = ['user_name', 'delta_first_last']
df_us_users = df_us_users.merge(user_delta, on = 'user_name')
df_us_users

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last
0,1131,188,okcnittany.144868,OKCNittany,1.182766e+09,"United States, Oklahoma",2007-06-25,187,2397
1,732,14,byofb.537281,BYOFB,1.292065e+09,"United States, Wisconsin",2010-12-11,32,756
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193
3,1078,10,colsanders123.545441,ColSanders123,1.293620e+09,"United States, Georgia",2010-12-29,135,1077
4,914,890,jays2629.517433,JayS2629,1.287828e+09,"United States, Alabama",2010-10-23,101,244
...,...,...,...,...,...,...,...,...,...
2227,548,21,crafteyedoc.675112,CraftEyeDoc,1.336039e+09,"United States, North Carolina",2012-05-03,143,970
2228,630,23,pmoney.592770,pmoney,1.302862e+09,"United States, Illinois",2011-04-15,132,854
2229,637,48,aleckpa.541457,aleckpa,1.292843e+09,"United States, North Carolina",2010-12-20,275,1853
2230,1117,4,dinoblueshoes.607196,DinoBlueShoes,1.309342e+09,"United States, Maine",2011-06-29,196,1047


In [654]:
# We compute the maximum number of ratings the user has done in a single day

user_day = df_us_ratings[['user_name','date']].groupby(["user_name", pd.Grouper(freq = "D", key="date")], as_index = False).size()
user_day = user_day.groupby('user_name', as_index = False).apply(lambda usr : max(usr['size']))
user_day.columns = ['user_name', 'max_active_day']
df_us_users = df_us_users.merge(user_day, on = 'user_name')

In [655]:
# We compute the maximum number of ratings the user has done in a single month

user_month = df_us_ratings[['user_name','date']].groupby(["user_name", pd.Grouper(freq = "M", key="date")], as_index = False).size()
user_month = user_month.groupby('user_name', as_index = False).apply(lambda usr : max(usr['size']))
user_month.columns = ['user_name', 'max_active_month']
df_us_users = df_us_users.merge(user_month, on = 'user_name') 

In [656]:
# We also want to know how much time the users took for submitting different number of ratings.

def compute_diff_date(user):
    diff = {}
    day = 50
    diff[str(day)] = (user[user["rating_number"] == day-1]["date"].item() -\
                         user[user["rating_number"] == 0]["date"].item()).days
    return pd.Series(diff)

In [657]:
def compute_diff_date_last(user):
    diff = {}
    day = 450
    diff[str(day)] = (user[user["rating_number"] == 500-1]["date"].item() -\
                         user[user["rating_number"] == day]["date"].item()).days
    return pd.Series(diff)

In [658]:
# We focus on the 10% first reviews (more justification)
user_deltas_0_50 = df_us_ratings.groupby('user_name', as_index = False).apply(compute_diff_date)
df_us_users = df_us_users.merge(user_deltas_0_50, on = 'user_name')

In [659]:
# We focus on the 10% last reviews (more justification)
user_deltas_450_500 = df_us_ratings.groupby('user_name', as_index = False).apply(compute_diff_date_last)
df_us_users = df_us_users.merge(user_deltas_450_500, on = 'user_name')

In [660]:
df_us_users.head(5)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last,max_active_day,max_active_month,50,450
0,1131,188,okcnittany.144868,OKCNittany,1.182766e+09,"United States, Oklahoma",2007-06-25,187,2397,11,91,45,16
1,732,14,byofb.537281,BYOFB,1.292065e+09,"United States, Wisconsin",2010-12-11,32,756,232,273,447,146
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193,12,42,64,335
3,1078,10,colsanders123.545441,ColSanders123,1.293620e+09,"United States, Georgia",2010-12-29,135,1077,238,247,254,159
4,914,890,jays2629.517433,JayS2629,1.287828e+09,"United States, Alabama",2010-10-23,101,244,20,103,21,15


In [661]:
# We now have a lot more information about the reviewing behaviour of our us users
# We proceed with some more filtering to get the final set of users on which we will conduct our study.

# Minimum number of active days required:
min_active_days = 100
df_valid_users = df_us_users[df_us_users['active_days'] >= min_active_days]

# How many days needed to submit their 500 ratings:
min_delta = 365
max_delta = 7*365
df_valid_users = df_valid_users[(df_valid_users['delta_first_last'] > min_delta) & (df_valid_users['delta_first_last'] < max_delta)]
# Maximum number of reviews in a single day: 
# On utilise pas pour le moment
max_per_day = 70

#df_valid_users = df_valid_users[df_valid_users['max_active_day'] < 70]
# Maximum number of reviews in a single month:
max_per_month = 70
df_valid_users = df_valid_users[df_valid_users['max_active_month'] <= max_per_month]

In [662]:
# To make the people more similar at the beginning and the end of their ratings career, 
# we decide to apply a last filter on the users

In [663]:
df_valid_users.head(5)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last,max_active_day,max_active_month,50,450
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193,12,42,64,335
5,676,676,dodo2step.110880,Dodo2step,1.165576e+09,"United States, Pennsylvania",2006-12-08,411,2153,13,28,45,442
7,653,651,kadonny.156863,Kadonny,1.188986e+09,"United States, Pennsylvania",2007-09-05,357,1800,5,22,249,263
8,748,748,biggred1.206640,biggred1,1.206616e+09,"United States, Indiana",2008-03-27,318,819,5,37,102,69
9,546,484,brownbeer.220672,brownbeer,1.211105e+09,"United States, Missouri",2008-05-18,143,1815,17,62,204,1302


In [674]:
df_check = df_valid_users['50'].to_frame()
df_check.sort_values(by = '50', inplace=True)
fig = px.line(df_check, x = range(df_check.shape[0]), y = '50')
fig.add_hline(500, line_color = 'red')
fig.show()

In [665]:
max_time_to_rate = 500
df_valid_users = df_valid_users[(df_valid_users['50'] < max_time_to_rate)]

In [678]:
df_check = df_valid_users['450'].to_frame()
df_check.sort_values(by = '450', inplace=True)
fig = px.line(df_check, x = range(df_check.shape[0]), y = '450')
fig.add_hline(500, line_color = 'red')
fig.show()

In [667]:
# We recut at 500
max_time_to_rate = 500
df_valid_users = df_valid_users[(df_valid_users['450'] < max_time_to_rate)]

In [668]:
# We have the final data set of users and the data set of their ratings

valid_usernames = set(df_valid_users['user_name'])
df_valid_ratings = df_us_ratings[df_us_ratings['user_name'].isin(valid_usernames)]

In [669]:
df_valid_users

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last,max_active_day,max_active_month,50,450
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193,12,42,64,335
5,676,676,dodo2step.110880,Dodo2step,1.165576e+09,"United States, Pennsylvania",2006-12-08,411,2153,13,28,45,442
7,653,651,kadonny.156863,Kadonny,1.188986e+09,"United States, Pennsylvania",2007-09-05,357,1800,5,22,249,263
8,748,748,biggred1.206640,biggred1,1.206616e+09,"United States, Indiana",2008-03-27,318,819,5,37,102,69
10,614,614,whynot44.2033,whynot44,1.052129e+09,"United States, Missouri",2003-05-05,276,1452,7,57,24,316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,707,0,gerred.720103,Gerred,1.361704e+09,"United States, Colorado",2013-02-24,203,924,38,57,63,72
2225,776,292,smanson56.782386,smanson56,1.392462e+09,"United States, New Hampshire",2014-02-15,250,653,8,48,100,44
2226,541,69,tamparainman.782764,TampaRainman,1.392635e+09,"United States, Florida",2014-02-17,140,666,21,56,76,98
2228,630,23,pmoney.592770,pmoney,1.302862e+09,"United States, Illinois",2011-04-15,132,854,39,50,26,114


In [670]:
df_valid_ratings

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number
1640018,167,Witbier,NaN,2001-03-16,frank4sail,4.50,4.50,4.00,5.00,5.00,4.75,Wheat Beers,0
311428,3558,Czech Pilsener,5.00,2001-06-18,Mark,4.00,4.00,3.00,4.00,4.00,3.90,Pale Lagers,0
17313,73,Milk / Sweet Stout,5.20,2001-06-18,Mark,5.00,5.00,5.00,5.00,5.00,5.00,Stouts,1
136697,261,Vienna Lager,4.48,2001-06-24,Mark,4.00,3.00,3.00,3.00,3.00,3.06,Dark Lagers,2
442671,87,American Pale Ale (APA),5.40,2001-06-24,Mark,4.00,4.00,1.00,3.00,3.00,3.10,Pale Ales,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089585,280331,American IPA,5.50,2017-07-20,greencovesteve,3.50,3.50,3.50,3.50,3.50,3.50,India Pale Ales,498
1082756,288077,American IPA,5.50,2017-07-21,greencovesteve,4.00,4.00,4.00,4.00,4.00,4.00,India Pale Ales,499
1194936,243002,German Pilsener,5.10,2017-07-22,coopdog1000,4.00,4.00,3.50,3.75,3.75,3.80,Pale Lagers,497
225295,213243,American Wild Ale,5.70,2017-07-22,coopdog1000,3.75,4.25,3.75,3.75,3.75,3.87,Wild/Sour Beers,498


In [671]:
# Begin different plots!

users_melt = pd.melt(df_us_users, id_vars=["user_name"], value_vars=["25", "50", "75", "100", "125", "150", "500"])
users_melt.columns = ["user_name", "cat_nb_days","day_diff"]
users_melt

KeyError: "The following 'value_vars' are not present in the DataFrame: ['100', '125', '150', '25', '500', '75']"

In [ ]:
fig = px.box(users_melt, x="cat_nb_days", y="day_diff")
fig.show()
#fig.write_html("day_diff.html")

### cluster

In [ ]:
df_to_cluster = df_reviews_final[df_reviews_final["rating_number"] < 50]

In [ ]:
user_to_cat = df_to_cluster.groupby(["user_name", "category"])["beer_id"].count()
user_to_cat = user_to_cat.unstack(level=-1)
user_to_cat.fillna(0, inplace = True)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
our_clustering = KMeans(4).fit(user_to_cat.values)

In [ ]:
import collections
# for cluster_num in range(4)    
#     np.count_nonzero(our_clustering.labels_ == 0)
count = collections.Counter(our_clustering.labels_)
count

Counter({2: 289, 0: 473, 1: 223, 3: 121})

In [ ]:
df_cluster_beer = pd.DataFrame(our_clustering.cluster_centers_.T)
#df_cluster_beer.rename(columns = user_to_cat.columns)
df_cluster_beer.index = user_to_cat.columns
df_cluster_beer["style"] = df_cluster_beer.index
df_cluster_beer = pd.melt(df_cluster_beer, id_vars=["style"], value_vars=[0,1,2,3])
df_cluster_beer.columns = ["style", "cluster", "Number of beer"]
df_cluster_beer

,style,cluster,Number of beer
0,Bocks,0,1.758985
1,Brown Ales,0,3.412262
2,Dark Ales,0,1.830867
3,Dark Lagers,0,1.824524
4,Hybrid Beers,0,0.331924
5,India Pale Ales,0,7.177590
6,Pale Ales,0,9.568710
7,Pale Lagers,0,4.395349
8,Porters,0,2.338266
9,Specialty Beer,0,1.676533


In [ ]:
fig = px.line(df_cluster_beer, x="style", y="Number of beer", color="cluster", title="Centroids obtained with Kmeans (k=4) of the first 50 ratings of each user")
#fig = px.line(df_cluster_beer, x="style", y=1)
fig.show()
fig.write_html("centroid.html")